# Exploring the Bias Variance Tradeoff

Author: Lukas Heinrich

Mar 2023

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

## Step 1: Write a function `generate_data(N)` 

Write a function `generate_data(N)` that produces `N` samples from the following model:

$$
p(s) = p(x,y) = p(y|x)p(x)
$$

with the following "true" underlying polynomial noisy model

$$p(x) = \mathrm{Uniform}(-1,1)$$
$$p(y|x) = \mathrm{Normal}(\mu = f(x),\sigma = 0.2)$$
$$f(x) = \sum_i p_i x^i$$,

with $p_0 = -0.7, p_1 = 2.2, p_2 = 0.5, p_3 = 1.0$

Hint: you can use `np.polyval` to evaluate a polynomial with a fixed set of coefficients (but watch out for the order)

The function should return a array of `x` values and an array of `y` values

In [ ]:
coeffs_true = [ ... ] # fill in

def generate_data(N):
    
    raise NotImplementedError
    
    return x,y

## Step 2: Plot Samples and Functions

Write a function `plot(ax, train_x, train_y, p_trained, p_true)` that
takes a matplotlib axis object and plots

* plot the true function 
* plot a second (trained or random) function 
* plot the samples 

In [ ]:
def plot(ax, train_x, train_y, p_trained, p_true):

    raise NotImplementedError

Check your function:

In [ ]:
f = plt.figure()
x,y = generate_data(10)
plot(f.gca(),x,y,np.random.normal(size = (4,)), coeffs_true)

## Step 3

One can show that given a Hypothesis Set of Polynomial functions

$$f(x) = \sum_i w_i x^i$$

and a risk function of the following form

$$l(s) = l(x,y) = (y - f(x))^2$$

there is a closed form solution for finding the empirical risk minimization, where the best fit coefficients $\vec{w}$ is given by

$$
w = (X^T X)^{-1} X^T y
$$

where $X$ is the matrix with rows $x = (x_0,x_1,x_2,x_3,\dots,x_d)$ and one row for each sample

$$
X = \left(
\begin{array}{}
x_0^{(1)},\dots,x_d^{(1)}  \\
x_0^{(2)},\dots,x_d^{(2)}  \\
\dots \\
x_0^{(n)},\dots,x_d^{(n)}  \\
\end{array}
\right)
$$

* Write a function `learn(train_x, train_y, degree)` to return the $(d+1)$ optimal coefficients for a polynomial fit of degree $d$.
* Fit a sampled of 5 data points with degree 4
* Plot the Trained function together with the true function using the plotting method from the last step
* Try this multiple time to get a feel for how much the data affects the fit
* Try degree 1 and observe how the trained function is much less sensitive to the data

In [ ]:
def learn(train_x, train_y, degree):
    raise NotImplementedError

## Step 4

Write a function to evaluate the risk or loss of a sample. Use our loss function for which we have the training procedure above

$$
l(s) = l(x,y) = (f(x) - y)^2
$$

and right a function `risk(x,y_true, trained_coeffs)` to compute

$$
\hat{L} = \frac{1}{N}\sum_i l(s_i) = \frac{1}{N}\sum_i l(x^{(i)},y^{(i)}) = \frac{1}{N}\sum_i ( f(x^{(i)}) - y^{(i)})^2
$$

* Draw a size 10 data sample and fit the result to obtain trained coefficients
* Draw 10000 samples of size 10 and compute their empirical risk under the trained coefficients
* Repeat the same but use the true coefficients of the underlying data-generating process
* Histogram the two sets of 10,000 risk evaluations. Which one has lower average risk?

In [ ]:
def risk(x, y_true, p):
    raise NotImplementedError

## Step 5

Explore how the fit improves when adding more data. Plot the best fit model for data set sizes of 

$$N = 5,10,100,200,1000$$

## Step 6

Explore how the fit changes when using more and more complex models. Plot the best fit model for degrees

$$d = 1,2,5,10$$

## Step 7 Bias-Variance Tradeoff

Draw two datasets:

* A train dataset with $N=10$
* A test dataset with $N=1000$

Perform trainings on the train dataset for degrees $1\dots8$ and store the training coefficients

* Evaluate the risk under the various trainings for the train and the test dataset
* Plot the train and test risk as a function of the polynomial degree

# Double Descent in Bias-Variance Tradeoff in Cubic Spline Regression

A -- at first sight unintuitive  -- result in Deep Learning specifically is that we often have highly overparametrized models but that those models still generalize well.

This means that the model has many more parameters than we have data points. The classic discussion
of the bias-variance tradeoff would indicate that in such overly complex models one would expect bad generalization.

The lore is that "with enough parameters I can fit anything, no? Once I have as many parameters as data points, 
I can essentially pick the one solution that goes through all data points. This will typicall generalize horribly!

But, surprisingly, the models that have even more parameters than that generalize well! How can this be?

What we observe is the phenomenon of "double descent" - the idea that the test error will
typically increase as we go to more and more complex models, but then - once we have more parameters
than we have data, we enter a territory over underconstrained models. That means we have 
increasingly more modela that all perform equally well on the data (that means perfectly, since we 
can in-fact fit everything).

In the over-parametrized regime, we must actively choose a model out of the family of well-performing functions.

We can use this freedom to choose slighly better behaved functions, such as the "minimum norm" solution. I.e. the solution
that exhibits as little "wiggliness" as possible. With this we can make the test error decrease again
for overparametrized models, leading to the "double descent" phenomenon

In a simple case without Deep Learning we can find the minimum norm solution (via singular-value decomposition) model by hand, as shown in this notebook,
but how does it work in Deep Learning?

Current research indicates that a key ingredient is the choice of optimization algorithm. Gradient Descent
appears to have a implicit regularization property that seeks out minimum-norm solution in the manifold of "interpolating models"

Check out the followint explainer Twitter thread by statistician Daniela Witten 

https://twitter.com/daniela_witten/status/1292293102103748609

and the paper

https://arxiv.org/abs/1912.02292


In [ ]:
from patsy import dmatrix

np.random.seed(10)

def true_function(x):
    return 3*np.sin(x)

# Generate random x values in the interval [0, 10]
x = np.random.uniform(0, 10, 15)
y = true_function(x) + np.random.normal(0, 3.0, len(x))

# Split the dataset into train and test sets (70% train, 30% test)
split_idx = int(len(x) * 0.7)
x_train, x_test = x[:split_idx], x[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# compute_errors

print(x_train.min())
print(x_test.min())
print(len(x_train))

def train_model(spline_basis_train, y_train, p):
    if p + 1 < len(x_train):
        coefficients = np.linalg.solve(spline_basis_train.T @ spline_basis_train, spline_basis_train.T @ y_train)
    else:
        U, s, Vt = np.linalg.svd(spline_basis_train, full_matrices=False)
        s_inv = np.zeros_like(s)
        s_inv[s != 0] = 1 / s[s != 0]
        spline_basis_pinv = Vt.T @ np.diag(s_inv) @ U.T
        coefficients = spline_basis_pinv @ y_train
    return coefficients

def compute_errors(x_train, y_train, x_test, y_test, p):
    '''
    TO DO: Your code here
    '''
    raise NotImplemented Error
    return train_mse, test_mse


In [ ]:
# Calculate train and test errors for different values of p
p_values = np.arange(3, 2*len(x_train))
errors = # To do, call compute_errors to calc this

# Plot train and test errors as a function of p
plt.plot(# YOUR CODE
         label='Train MSE', marker='o')
plt.plot(# YOUR CODE
         label='Test MSE', marker='o')
plt.xlabel('Number of Parameters (p)')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid()
plt.title('Train and Test Errors as a Function of p')
plt.ylim(1e-11,1e4)
plt.semilogy()
# plt.ylim(0,0.02)
plt.show()


**What do you see?? Does it make sense?**

In [ ]:
def fit_and_plot():
    # Generate training data
    x_train = np.random.uniform(0, 10, 15)
    y_train = true_function(x_train) + np.random.normal(0, 3.0, len(x_train))

    # Define models
    model_p_values = [
        2*len(x_train),  # Overparametrized model
        len(x_train),    # Interpolation threshold model
        len(x_train)//2  # Underparametrized model
    ]

    # Generate x values for ground truth function and fitted functions
    x_values = np.linspace(0, 10, 1000)
    y_true = true_function(x_values)

    # Plot training data and ground truth function
    plt.scatter(x_train, y_train, c='black', marker='o', label='Training data')
    plt.plot(x_values, y_true, linestyle='--', label='Ground truth function')

    for p in model_p_values:
        common_min = max(x_train.min(), x_values.min())
        common_max = min(x_train.max(), x_values.max())
        knots = np.linspace(common_min + (common_max - common_min) * 0.01,
                            common_max - (common_max - common_min) * 0.01, p - 2)
        
        spline_basis_train = dmatrix("cr(x, knots=knots) - 1", {"x": x_train, "knots": knots})
        spline_basis_values = dmatrix("cr(x, knots=knots) - 1", {"x": x_values, "knots": knots})

        coefficients = train_model(spline_basis_train, y_train, p)
        y_fitted = spline_basis_values @ coefficients

        plt.plot(x_values, y_fitted, label=f'Fitted function (p={p})')

    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.ylim(-20,20)
    plt.grid()
    plt.title('Training Data, Ground Truth Function, and Fitted Functions')
    plt.show()

fit_and_plot()


**Q:** Which model has the best fit? which has the worst fit?